# Get ready

First, download, import, prep packages and such. 

Then, check the file location and import the .csv files. Remove any with empty text fields. 

Save a data frame with all the texts and another with only those texts that mention the keywords of interest. 

In [1]:
%%capture

# installing necessary pdf conversion packages via pip
# the '%%capture' at the top of this cell suppresses the output (which is normally quite long and annoying looking). 
# You can remove or comment it out if you prefer to see the output. 
!pip install nltk
!pip install autocorrect        
!pip install pyspellchecker 
!pip install spacy -q
!python -m spacy download en_core_web_lg -q


In [2]:
import os                         # os is a module for navigating your machine (e.g., file directories).

import nltk                       # nltk stands for natural language tool kit and is useful for text-mining. 
from nltk import word_tokenize    # and some of its key functions
from nltk import sent_tokenize  
tokenizer = nltk.tokenize.punkt.PunktSentenceTokenizer()
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from nltk.corpus import wordnet                    # Finally, things we need for lemmatising!
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer() 
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
nltk.download('averaged_perceptron_tagger')        # Like a POS-tagger...
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('webtext')
from nltk.corpus import webtext

import pandas as pd
pd.set_option('display.max_colwidth', 200)
import numpy as np
import statistics
import datetime
date = datetime.date.today()

from autocorrect import Speller   # things we need for spell checking
check = Speller(lang='en')
import codecs
import csv                        # csv is for importing and working with csv files

from collections import Counter

import statistics
import re                         # things we need for RegEx corrections
import matplotlib.pyplot as plt
import string 
import spacy 
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from spacy import displacy 
nlp = spacy.load('en_core_web_lg')
nlp.max_length = 1500000 #or any large value, as long as you don't run out of RAM

import math 

English_punctuation = "-!\"#$%&()'*-–+,./:;<=>?@[\]^_`{|}~''“”"      # Things for removing punctuation, stopwords and empty strings
table_punctuation = str.maketrans('','', English_punctuation)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mzyssjkc\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already u

In [3]:
print(os.listdir("..\\results")  )

files = []
def import_results(input):
    for f in os.listdir(input):
        f = pd.read_csv(input + '\\'+ f,encoding='latin1')
        files.append(f)
    output = pd.concat(files)
    return output

['ESHG2001abstractICHG.csv', 'ESHG2002Abstracts.csv', 'ESHG2003Abstracts.csv', 'ESHG2005Abstracts.csv', 'ESHG2006Abstracts.csv', 'ESHG2007Abstracts.csv', 'ESHG2008Abstracts.csv', 'ESHG2009Abstracts.csv', 'ESHG2010Abstracts.csv', 'ESHG2011Abstracts.csv', 'ESHG2012Abstracts.csv', 'ESHG2013Abstracts.csv', 'ESHG2014Abstracts.csv', 'ESHG2015Abstracts.csv', 'ESHG2016Abstracts.csv', 'ESHG2016Abstractscleaned.csv', 'ESHG2017 electronic posters.csv', 'ESHG2017 oral presentation.csv', 'ESHG2018 electronic posters.csv', 'ESHG2018 oral presentation.csv', 'ESHG2019 electronic posters.csv', 'ESHG2019 oral presentation.csv', 'ESHG2020 electronic posters.csv', 'ESHG2020 oral presentation.csv', 'ESHG2021 electronic posters.csv', 'ESHG2021 oral presentation.csv']


In [95]:
all_results = import_results("..\\results")
len(all_results)

69178

In [96]:
no_null_texts = all_results[~all_results['Text'].isnull()]
len(no_null_texts)

67550

In [97]:
matched_texts = no_null_texts[no_null_texts['Text'].str.contains('autis|Autis|ASD|Asperger|asperger')]
len(matched_texts)

1948

# Sent tokenisation

In [99]:
sentences  = [sent_tokenize(abstract) for abstract in matched_texts['Text'] ] # make tokenized sentences list
matched_texts['Sentence'] = sentences                                         # copy that list back into df as a new column
matched_texts = matched_texts.explode('Sentence')                             # explode to create 1 row per sentence token


C:\Users\mzyssjkc\AppData\Local\Temp\ipykernel_20720\1360292582.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_texts['Sentence'] = sentences                                         # copy that list back into df as a new column


1948

In [101]:
matched_sents = matched_texts[matched_texts['Sentence'].str.contains('autis|Autis|ASD|Asperger|asperger')]
                                                                            # keep only those rows that contain the keywords
matched_sents = matched_sents.drop_duplicates()                             # drop any duplicates
len(matched_sents)                                                          # check length of remaining data frame

4500

In [112]:
def prep_sent_for_pattern_match(input):                                       # define a new function
    spacy_objects = []                                                        # creates an empty list
    for sentence in input:                                                    # then takes string objects from the input 
        print(sentence)
        sentence = nlp(sentence)                                              # converts them to SpaCy objects 
        spacy_objects.append(sentence)                                        # appends them to the empty list
    return spacy_objects                                                  # and returns the list


In [114]:
matched_sents['spacy_objects'] = prep_sent_for_pattern_match(matched_sents['Sentence'])  #create new column with spacy objects

syndactyly  heart and lungdefects)  failure to thrive and mental retardation or autism.
Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy  whose clinical features resembled autism with some modifications.
Itis characterised by difficulties in social interaction and stereotyped behav iour but in contrast to infantile autism by normal intelligence and usuallynormal language skills.
In the pediatric age group the prevalence ofAsperger syndrome is around 4 7/1000 whereas the corresponding figurefor the infantile autism is 4/10000.
Based on clinical observationsAS is inherited in the same families as infantile autism but also as a sepa rate entity often transmitted from male to male.
We report the analysis of13 candidate gene loci associated with autism and schizophrenia in 17Finnish AS families with autosomal dominant mode of inheritance.
Linkageto the previously reported predisposing loci for a

One of them has an autistic disorder with severe MR an a dysmor  nan
Comprehensive use of hands  microcephaly  seizures  autistic behavior  ataxia  hyperven  ND assessments were completed using standardized tools to determine tilation and stereotypic hand movements.
Speech(EL=85.7[sd=12.8] p<.001)& motor(PDI= movements  (2/4) seizures  (2/4) loss of walking (0/4) microcephaly  (2/4) 88.46[sd=12.39] p<.001)delays were significant with normal autistic behavior (2/4) ataxia and (1/4) hyperventilation.
Characteristic vertebral cognitive and adaptive regression with autistic features  loss of acquired shapes that has not been described in the medical literature before was skills like speech and hand usage  stereotypic hand movements  epilepsy  also found.
 for multiply malformed children (for example four times higher for ASD).
In conclusion  we provided that characterized by several clinical abnormalities  such as progressive DHPLC analysis is a sensitive and specific method to detect muta

The patients show a number of  clinical signs indicative of a neurodevelopmental defect : arrest of  brain development, regression of acquisitions, and behavioural  troubles (stereotypic hand movement, autism).
Rett syndrome (RTT) and autism (A) are classified as separate  disorders.
Recently, the identification of the Preserved Speech  Variant (PSV), where autistic behavior is usual, stressed the  phenotypic overlap between these two conditions.
On  the other hand, we and others have provided evidence that 15q11  q13 maternally inherited duplications are found in autistic children,  suggesting that an abnormal dosage of gene(s) within this region  might be responsible for A.
We have examined a girl  with a balanced translocation t(X:10)(p11.4:q11.1) associated with  severe MR, autism and epilepsy.
The findings may further clarify the role of different TM4SF2  isoforms  in severe MR, autism and epilepsy.
She had a small ASD; brain CT was normal.
Additional features may include obesity,

It can be used on any DNA source (blood, amniocytes, CVS, parafÃ¯Â¬Ân embedded tissue Ã¢ÂÂ¦)  We used 2 commercial sets of probes, the SALSA P019 combined to P020 and the P036 human telomere MLPA kits (MRC Holland, Amsterdam, Netherlands) to assay retrospectively patients with known dup/del syndromes, and to test a group of more than 100 systematically collected patients referred for evaluation of MR or autism.
Autism is a severe neurodevelopmental disorder that usually occurs due to a complex genetic predisposition.
Previously linkage for autism has been found to chromosome 7q, which has been replicated by many groups.
Four candidate genes present in chr7q22 31 have been screened for mutations and association with autism in the IMGSAC sample; RELN, LAMB1, CUTL1  and NRCAM.
Atrial septum defect (ASD) is a common congenital heart defect affecting  4 in 100,000 individuals.
ASD of the secundum type is characterised by an incomplete coverage of the ostium secundum resulting in a left to

Pedigree analysis is consist with an autosomal dominant inheritance of the disease .Genotyping of the ASD family with  2 microsatellite markers D5S394 and D5S2439 flanking Nkx2.5 gene  and Linkage analysis showed exclusion of linkage between the gene  responsible for ASD in this family and Nkx2.5  gene, thus confirming  genetic heterogeneity of this phenotype .
Autism and ADHD are frequently observed in part of the DS .
Glutamate receptor signalling has previously been implicated in the  pathogenesis of the fragile X syndrome and may also play a role in  autism.
His language is severely delayed,  and he has autistic like features.
Yatsenko et.al (2004) re   viewed 19 patients with deletion in 10p and congenital heart defects  (CHD) and found that atrial septum defect (ASD) is a common cardiac  anomaly associated with
We present a boy who was diagnosed to have Asperger autism at the  age of 19 years.
Afterwards he tried to graduate from expert school for social sciences which could not 

...

Sleep is disturbed in upto 80% of affected youths with ASD.
Genes underlying the circadian rhythm are proposed to elucidate sleep and other timing problems in individuals with ASD.
1423signiï¬cantly overrepresented in ASD individuals compared to their unaffected siblings.
Material and Methods: We used microarray data from probands with ASD (n ,2926) and their unaffected siblings (n,2434) from the Simons Simplex Collection (SSC).
Results: Fisher âs exact test reveal that CNVs encom  passed in core circadian clock genes, circadian pathway genes, and sleep trait candidate genes are signi ï¬cantly overrepresented in probands with ASD in comparison totheir unaffected siblings (p ,0.003; Odds ratio 1.6; CI95% [1.2 ; 2.3]).
Conclusions: These preliminary results are the ï¬rst to implicate a broad network of sleep related genes in the etiology of ASD.
Furthermore, SNP set Kernel Association Test (SKAT) using probands â WES and corresponding phenotypic sleep observations will identify ca

Obregia Clinical Hospital of Psychiatry, Department of Pediatric Neurology, Bucharest, Romania,4Cytogenomic Medical Laboratory, Bucharest, Romania,5Titu Maiorescu Uni  versity, Faculty of Medicine, Bucharest, Romania Copy number variants (CNVs) have been shown to be involved in etiology of neurodevelopmental disorders like autism spectrum disorders (ASDs), intellectual disability(ID), and other psychiatric disorders.
Chromosomal microarray analyses (CMA) has offered a high detection rate of small and large CNVs associated with ASD, and ledto discovery of new autism genes.
We report on the results of chromosomal CMA investigation in a group of ASDs patients.
Eighty patients were referred to our laboratory forgenetic testing with ASDs as part of a broad phenotype including ID or other clinical features.
A complete clinical evaluation was performed with focus on psychiatric exam ination and psychological evaluation with speci ï¬c ASDs tests (ADOS, ADI R).
Besides well described syndromic

In humans, mutations in UPF3B cause intellectualdisability (ID) and are strongly associated with autism, atten  tion de ï¬cit hyperactivity disorder, and schizophrenia (SCZ).
When small head circumference re ï¬ects an insufï¬cient brain growth, microcephaly is associated with neurodevelopmental disorders (NDD), such as intellectualdisability, epilepsy, autism.
Shear2, G. Ruggeri2, E. Novotny3, W. Dobyns2, R. Kapur3, J. Hauptman2, J. Ojemann2 1Seattle, WA, United States,2Seattle Children âs Research Institute, Seattle, WA, United States,3Seattle Children âs Hospital, Seattle, WA, United States Megalencephaly: advances from genomic discoveries to precision based medicine Megalencephaly disorders (MEG) constitute a growing spectrum of developmental disorders associated with early brain overgrowth, brain malformations including corticaldysplasia, and neurodevelopmental co morbidities such as intractable epilepsy, autism and intellectual disability.
Fisher1,3 1Max Planck Institute fo

Autism spectrum disorder (ASD) is a neurodevelopmental condition characterized by social impairment, restricted interests and repetitive behaviors.
ID and ASD symptoms are oftenoverlapping.
In the present study, we investigated by Whole ExomeSequencing (WES) analysis a total of 200 ID/ASD patients.
Our cohort included 40 patients with syndromic ID, 64 with non syndromic ID, 6 with autism and syndromic ID, 19 with autism and non syndromicID, and71 with isolated autism.
In particular, 2 9P V sw e r ef o u n di nI Dp a t i e n t sa n d3i nA S Dp a t i e n t s .7P V sw e r e identi ï¬ed in patients with ID and ASD features.
In conclusion, our results demonstrated the ef ï¬ciency of WES analysis on the identi ï¬cation of PVs in ID/ASD patients.
Depending on whether a deletion or a duplication of17p13.3 occurs, different rare neurodevelopmental disorders arise.Phenotypic features of 17p13.3 microduplication syndrome (MIM #613275) include developmental and psychomotor delay, beha vioral pr

I will show howforms of both inherited and sporadic mutations have helped toidentify new genes associated with both syndromic and non syndromic forms of autism.
The available data suggest that multiple rare, privately inherited genetic variants contributesigni ï¬cantly to autism disease risk.
We andothers have identi ï¬ed highly speci ï¬c and highly sensitive genome wide DNAm signatures for several of these epigenes including DNMT3A, NSD1, EZH2/EED, and CHD8; mutations inthese genes are associated with Tatton Brown Rahman, Sotos,Weaver, and Autism syndromes, respectively.
Results: The clinical pro ï¬le of this disease includes develop mental delay/intellectual disability, autism spectrum disorder, anxiety, aggressive behavior, attention de ï¬cit disorder, hypotonia, brain and spine anomalies, congenital heart defects, high/narrowpalate, facial dysmorphisms, and obesity/increased BMI, especiallyin females.
Wefurther applied LT FH ++ to mortality in the UK biobank and four common ps

In [117]:
matched_sents.head()

,Title,Session_Code,Authors_and_Affiliations,Text,Year,Email,Author,Affiliations,Sentence,spacy_objects
59,Genetic defects in sterol metabolism,S60.,F. MoebiusInstitute for Biochemical Pharmacology University of Innsbruck Innsbruck AustriaFabian.M,Genetic defects in sterol metabolizing enzymes have recently emerged asimportant causes of dysmorphogenetic syndromes. They affect enzymesrequired for the removal of methyl groups at C4(NSHDL) th...,2001,Fabian.Moebius@uibk.ac.at,NaN,NaN,syndactyly heart and lungdefects) failure to thrive and mental retardation or autism.,"(syndactyly, , heart, and, lungdefects, ), , failure, to, thrive, and, mental, retardation, or, autism, .)"
150,Location of the first predisposing gene locus for Asperger syndrome on chromosome 1q21 22,C102.,E. Jarvela1 T. Ylisaukko oja2 T. Nieminen3 E. Kempas1 M. Auranen1 L. Peltonen1 1National Public Health Institute Helsinki Finland 2National Public Health Insitute Helsinki Finland 3Uni...,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,2001,irma.jarvela@hus.fi,NaN,NaN,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,"(Asperger, syndrome, (, AS, ), was, first, described, in, 1944, by, a, Viennese, physi, cian, Hans, Asperger, who, reported, a, group, of, boys, with, autistic, psychopa, thy, , whose, clinical, ..."
150,Location of the first predisposing gene locus for Asperger syndrome on chromosome 1q21 22,C102.,E. Jarvela1 T. Ylisaukko oja2 T. Nieminen3 E. Kempas1 M. Auranen1 L. Peltonen1 1National Public Health Institute Helsinki Finland 2National Public Health Insitute Helsinki Finland 3Uni...,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,2001,irma.jarvela@hus.fi,NaN,NaN,Itis characterised by difficulties in social interaction and stereotyped behav iour but in contrast to infantile autism by normal intelligence and usuallynormal language skills.,"(Itis, characterised, by, difficulties, in, social, interaction, and, stereotyped, behav, iour, but, in, contrast, to, infantile, autism, by, normal, intelligence, and, usuallynormal, language, sk..."
150,Location of the first predisposing gene locus for Asperger syndrome on chromosome 1q21 22,C102.,E. Jarvela1 T. Ylisaukko oja2 T. Nieminen3 E. Kempas1 M. Auranen1 L. Peltonen1 1National Public Health Institute Helsinki Finland 2National Public Health Insitute Helsinki Finland 3Uni...,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,2001,irma.jarvela@hus.fi,NaN,NaN,In the pediatric age group the prevalence ofAsperger syndrome is around 4 7/1000 whereas the corresponding figurefor the infantile autism is 4/10000.,"(In, the, pediatric, age, group, the, prevalence, ofAsperger, syndrome, is, around, 4, 7/1000, whereas, the, corresponding, figurefor, the, infantile, autism, is, 4/10000, .)"
150,Location of the first predisposing gene locus for Asperger syndrome on chromosome 1q21 22,C102.,E. Jarvela1 T. Ylisaukko oja2 T. Nieminen3 E. Kempas1 M. Auranen1 L. Peltonen1 1National Public Health Institute Helsinki Finland 2National Public Health Insitute Helsinki Finland 3Uni...,Asperger syndrome (AS) was first described in 1944 by a Viennese physi cian Hans Asperger who reported a group of boys with autistic psychopa thy whose clinical features resembled autism with som...,2001,irma.jarvela@hus.fi,NaN,NaN,Based on clinical observationsAS is inherited in the same families as infantile autism but also as a sepa rate entity often transmit

In [125]:
test = matched_sents['spacy_objects']

test

59                                                                                                (syndactyly,  , heart, and, lungdefects, ),  , failure, to, thrive, and, mental, retardation, or, autism, .)
150    (Asperger, syndrome, (, AS, ), was, first, described, in, 1944, by, a, Viennese, physi, cian, Hans, Asperger, who, reported, a, group, of, boys, with, autistic, psychopa, thy,  , whose, clinical, ...
150    (Itis, characterised, by, difficulties, in, social, interaction, and, stereotyped, behav, iour, but, in, contrast, to, infantile, autism, by, normal, intelligence, and, usuallynormal, language, sk...
150                             (In, the, pediatric, age, group, the, prevalence, ofAsperger, syndrome, is, around, 4, 7/1000, whereas, the, corresponding, figurefor, the, infantile, autism, is, 4/10000, .)
150                    (Based, on, clinical, observationsAS, is, inherited, in, the, same, families, as, infantile, autism, but, also, as, a, sepa, rate, entity, often, tra

## Person-first pattern

In [120]:
pattern_2 = [{"POS": "NOUN"},
             {'LOWER': 'with'},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"TEXT": {"REGEX": "^[Aa]utism$"}}]

pattern_3 = [{"POS": "NOUN"},
             {'LOWER': 'with'},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"TEXT": {"REGEX": "^[Aa]sperger$"}}]

pattern_4 = [{"POS": "NOUN"},
             {'LOWER': 'with'},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"TEXT": {"REGEX": "^ASD$"}}]

# Matcher class object 
matcher = Matcher(nlp.vocab) 
matcher.add("matching_1", [pattern_2, pattern_3, pattern_4]) 

In [140]:
person_first = []
matches = []
for thingy in test:
    match = matcher(thingy)
    if match != []:
        matches.append(match)

len(matches)
        
    

187

In [145]:
person_first = []

for thingy in test:
    match = matcher(thingy)
    if match != []:
        for match_id, start, end in match:
            string_id = nlp.vocab.strings[match_id]  # Get string representation
            span = thingy[start:end]  # The matched span
            person_first.append(span.text)
    
print(person_first)        
    

['patients with autism', 'families with autism', 'individuals with autism', 'patient with high functioning autism', 'families with autism', 'individuals with idiopathic autism', 'individuals with autism', 'patients with autism', 'patients with autism', 'association with autism', 'patients with autism', 'families with ASD', 'patients with autism', 'patients with autism', 'individuals with autism', 'children with idiopathic autism', 'patient with autism', 'patients with autism', 'patients with autism', 'patients with autism', 'patient with autism', 'deletion with autism', 'sociation with autism', 'tients with ASD', 'association with autism', 'brothers with autism', 'patients with autism', 'patients with ASD', 'patients with autism', 'families with autism', 'patients with autism', 'brother with infantile autism', 'child with autism', 'patients with autism', 'patients with autism', 'patients with autism', 'patients with ASD', 'patients with idiopathic autism', 'children with autism', 'pati

In [141]:
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = test[start:end]  # The matched span
    person_first.append(span.text)
    
print(person_first)

ValueError: not enough values to unpack (expected 3, got 1)

In [129]:
for thingy in test:
    print(type(thingy))

<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class 'spacy.tokens.doc.Doc'>
<class '

In [ ]:
p_f_lower = [word.lower() for word in person_first]     # make those tokens lowercase
p_f_no_punct = [w.translate(table_punctuation) for w in p_f_lower] # remove the punctuation
p_f_no_space = (list(filter(lambda x: x, p_f_no_punct)))           # remove any extra whitespace


In [ ]:
person_first_no_dups = list(set(p_f_no_space))
with open('..\\counts\\ESHG\\person_first.csv', "w", encoding='ISO-8859-1') as outfile:
        write = csv.writer(outfile)
        for item in person_first_no_dups:
            write.writerow([item])

In [ ]:
print(len(p_f_no_space))
print(len(person_first_no_dups))

## Identity first pattern

In [ ]:
pattern_a = [{"TEXT": {"REGEX": "^[Aa]utistic"}},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"POS": "NOUN"}]

pattern_b = [{"TEXT": {"REGEX": "^[Aa]sperger"}},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"POS": "NOUN"}]

pattern_c = [{"TEXT": {"REGEX": "^ASD"}},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {'DEP':'amod', 'OP':"?"},
             {"POS": "NOUN"}]

In [ ]:
# Matcher class object 
matcher = Matcher(nlp.vocab) 
matcher.add("matching_1", [pattern_a, pattern_b, pattern_c]) 

identity_first =[]

In [ ]:
    
matches = matcher(p_i_doc) 
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = p_i_doc[start:end]  # The matched span
    identity_first.append(span.text)

#print(matches[:10])

In [ ]:
i_f_lower = [word.lower() for word in identity_first]     # make those tokens lowercase
i_f_no_punct = [w.translate(table_punctuation) for w in i_f_lower] # remove the punctuation
i_f_no_space = (list(filter(lambda x: x, i_f_no_punct)))           # remove any extra whitespace


In [ ]:
identity_first_no_dups = list(set(i_f_no_space))
with open('..\\counts\\ESHG\\identity_first.csv', "w", encoding='utf8') as outfile:
        write = csv.writer(outfile)
        for item in identity_first_no_dups:
            write.writerow([item])

In [ ]:
print(len(identity_first))
print(len(identity_first_no_dups))

## Word counts by part of speech


In [ ]:
POS_p_i = []

for token in p_i_doc:
    this_token = [token.text, token.lemma_, token.pos_, token.tag_]
    if any (s in token.text for s in ['autistic', 'Autistic', 'autism', 'Autism', 'ASD', 'asd', 'Asperger', 'asperger']):
        POS_p_i.append(this_token)

In [ ]:
with open('..\\counts\\ESHG\\POS.csv', "w", encoding='utf8') as outfile:
        write = csv.writer(outfile)
        for item in POS_p_i:
            write.writerow([item])

In [ ]:
p_f_lower = [word.lower() for word in person_first]     # make those tokens lowercase
p_f_no_punct = [w.translate(table_punctuation) for w in p_f_lower] # remove the punctuation
p_f_no_space = (list(filter(lambda x: x, p_f_no_punct)))           # remove any extra whitespace

In [ ]:
#for saving output
os.makedirs('folder/subfolder', exist_ok=True)  
df.to_csv('folder/subfolder/out.csv') 